In [1]:
# Author Silin Huang
# ID A20383068
# Reference: tutorial & elastic search documentation
import os
os.environ['PATH']
import pandas as pd                                    # panda's nickname is pd
import numpy as np                                     # numpy as np
from pandas import DataFrame, Series                   # for convenience
import json
import folium                                          # folium package
from folium import plugins
from elasticsearch import Elasticsearch, helpers       # elastic search
es = Elasticsearch()

In [2]:
issueList = [json.loads(line) for line in open('SPM587SP18issues.json')]

In [3]:
# Create the DataFrame object for the list_of_issues_dict_data object

issues_df = DataFrame(issueList)

In [4]:
# Prepare and Clean the dataframe object

wrangled_issues_df = issues_df[['Author','State','closed_at','created_at','issue_number','labels']]
wrangled_issues_df.loc[0:len(wrangled_issues_df), 'OriginationPhase']= np.NaN
wrangled_issues_df.loc[0:len(wrangled_issues_df),'DetectionPhase']= np.NaN
wrangled_issues_df.loc[0:len(wrangled_issues_df),'Category']= np.NaN
wrangled_issues_df.loc[0:len(wrangled_issues_df),'Priority']= np.NaN
wrangled_issues_df.loc[0:len(wrangled_issues_df),'Status']= np.NaN

In [5]:
for i in range(0, len(wrangled_issues_df)):
    if wrangled_issues_df.iloc[i]['labels']:
        for label in wrangled_issues_df.iloc[i]['labels']:
            label_name= (label.split(':'))[0]
            label_value= (label.split(':'))[1]
            wrangled_issues_df.loc[i, label_name]=label_value

In [6]:
s = wrangled_issues_df.to_json(orient = 'records')

In [7]:
cont = json.loads(s)

In [8]:
actions = []
for data in cont:
    action = {
        "_index": "issues",
        "_type": "issue",
        "_id": data['issue_number'],
        "_source": data
    }
    actions.append(action)
helpers.bulk(es, actions)

(255, [])

In [9]:
doc = {
    'size': 10000,
    'query': {
        'match_all': {}
    }
}
result_without_condition = es.search(index = 'issues', body = doc, scroll = '1h')
result_without_condition

{'_scroll_id': 'DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAAExFklKZEVBUFN6VEplWVVST29xekpZa1EAAAAAAAABLRZJSmRFQVBTelRKZVlVUk9vcXpKWWtRAAAAAAAAAS4WSUpkRUFQU3pUSmVZVVJPb3F6SllrUQAAAAAAAAEvFklKZEVBUFN6VEplWVVST29xekpZa1EAAAAAAAABMBZJSmRFQVBTelRKZVlVUk9vcXpKWWtR',
 '_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '470',
    '_index': 'issues',
    '_score': 1.0,
    '_source': {'Address': None,
     'Address ': None,
     'Author': 'RSP18SCM19N',
     'Category': 'Enhancement',
     'DetectionPhase': 'Testing',
     'Latitude': None,
     'Latitude ': None,
     'Lattitude': None,
     'Longitude': None,
     'Longitude ': None,
     'OriginationPhase': 'Design',
     'Priority': 'Major',
     'State': 'closed',
     'Status': 'Completed',
     'closed_at': '2018-04-14',
     'created_at': '2018-04-14',
     'issue_number': 470,
     'labels': ['Category:Enhancement',
      'DetectionPhase:Testing',
      'OriginationPhase:Design',
      'Priority:Major',
 

In [10]:
sid = result_without_condition["_scroll_id"]
scroll_size = result_without_condition["hits"]["total"]

In [11]:
count = 0
location_set = []
while (scroll_size > 0):
    print("Scrolling...", count)
    for doc in result_without_condition['hits']['hits']:
        location = []
        res = doc['_source']
        count = count + 1
        if 'Latitude' in res.keys():
            if 'Longitude' in res.keys():
                if 'Address' in res.keys():
                    if res['Latitude'] != None and res['Longitude'] != None and res['Address'] != None:
                        location.append(float(res['Latitude']))
                        location.append(float(res['Longitude']))
                        location_set.append(location)
    result_without_condition = es.scroll(scroll_id = sid, scroll = '5m')
    sid = result_without_condition['_scroll_id']
    scroll_size = len(result_without_condition['hits']['hits'])
print("count is: ", count)

Scrolling... 0
count is:  255


In [12]:
location_set

[[41.878693, -87.638924],
 [41.838897, -87.646804],
 [41.858415, -87.660926],
 [41.89302, -87.631556],
 [41.878693, -87.638924],
 [41.847095, -87.616767],
 [41.847095, -87.616767],
 [41.847095, -87.616767],
 [41.877817, -87.631247],
 [41.89323, -87.617419],
 [41.847095, -87.616767],
 [41.877846, -87.631296],
 [41.838897, -87.646804],
 [41.838897, -87.646804],
 [41.847095, -87.616767],
 [41.877846, -87.631296],
 [41.878693, -87.638924],
 [41.89323, -87.617419],
 [41.883772, -87.625962],
 [41.917164, -87.686965],
 [41.883772, -87.625962],
 [41.917164, -87.686965],
 [41.880982, -87.630553],
 [41.917164, -87.686965],
 [41.89323, -87.617419],
 [41.891551, -87.607375],
 [40.170101, -92.177847],
 [41.883772, -87.625962],
 [41.917164, -87.686965],
 [41.847095, -87.616767],
 [41.838897, -87.646804],
 [41.89302, -87.631556],
 [41.847095, -87.616767],
 [41.847441, -87.679408],
 [41.847095, -87.617419],
 [41.847095, -87.616767],
 [41.89302, -87.631556],
 [41.847095, -87.616767],
 [41.877817, -87.6

In [13]:
#Q3.1 Create Folium Heatmap to plot all issues retrieved from ElasticSearch on Chicago HeatMap 

q31_heatmap = folium.Map([41.90, -87.70], zoom_start = 11)
q31_heatmap.add_child(plugins.HeatMap(location_set[:40000], radius = 15))

In [14]:
doc321 = {
    'size' : 10000,
    'query': {
        'bool': {
            'must': [{'match': {'DetectionPhase': 'Field'}}, {'match': {'Priority': 'Critical'}}]
        }
    }
}
result321 = es.search(index = 'issues', body = doc321, scroll = '1h')

In [15]:
scroll_size321 = result321['hits']['total']
sid = result321["_scroll_id"]

In [16]:
count = 0
location_set321 = []
while (scroll_size321 > 0):
    print("Scrolling...", count)
    for doc in result321['hits']['hits']:
        location = []
        res = doc['_source']
        count = count + 1
        if 'Latitude' in res.keys():
            print("latitude can get")
            if 'Longitude' in res.keys():
                print("longitude can get")
                if 'Address' in res.keys():
                    print("address can get")
                    if res['Latitude'] != None and res['Longitude'] != None and res['Address'] != None:
                        print("got one")
                        location.append(float(res['Latitude']))
                        location.append(float(res['Longitude']))
                        location_set321.append(location)
    result321 = es.scroll(scroll_id = sid, scroll = '5m')
    sid = result321['_scroll_id']
    scroll_size321 = len(result321['hits']['hits'])
print("count is: ", count)

Scrolling... 0
latitude can get
longitude can get
address can get
latitude can get
longitude can get
address can get
got one
latitude can get
longitude can get
address can get
got one
latitude can get
longitude can get
address can get
got one
count is:  4


In [17]:
location_set321

[[41.89323, -87.617419], [41.917164, -87.686965], [41.891551, -87.607375]]

In [18]:
#Q3.2 1) DetectionPhase is Field AND Priority is Critical 

q321_heatmap = folium.Map([41.90, -87.70], zoom_start = 11)
q321_heatmap.add_child(plugins.HeatMap(location_set321[:40000], radius = 15))

In [19]:
doc322 = {
    'size' : 10000,
    'query': {
        'bool': {
            'must': [{'match': {'DetectionPhase': 'Field'}}, {'match': {'Status': 'Completed'}}]
        }
    }
}
result322 = es.search(index = 'issues', body = doc322, scroll = '1h')

In [20]:
scroll_size322 = result322['hits']['total']
sid = result322["_scroll_id"]

In [21]:
count = 0
location_set322 = []
while (scroll_size322 > 0):
    print("Scrolling...", count)
    for doc in result322['hits']['hits']:
        location = []
        res = doc['_source']
        count = count + 1
        if 'Latitude' in res.keys():
            print("getting latitude")
            if 'Longitude' in res.keys():
                print("getting longitude")
                if 'Address' in res.keys():
                    if res['Latitude'] != None and res['Longitude'] != None and res['Address'] != None:
                        print("got one")
                        location.append(float(res['Latitude']))
                        location.append(float(res['Longitude']))
                        location_set322.append(location)
    result322 = es.scroll(scroll_id = sid, scroll = '5m')
    sid = result322['_scroll_id']
    scroll_size322 = len(result322['hits']['hits'])
print("count is: ", count)

Scrolling... 0
getting latitude
getting longitude
got one
getting latitude
getting longitude
got one
getting latitude
getting longitude
got one
getting latitude
getting longitude
got one
count is:  4


In [22]:
location_set322

[[40.170101, -92.177847],
 [41.891551, -87.607375],
 [41.89323, -87.617419],
 [41.853136, -87.63316]]

In [23]:
#Q3.2 2) DetectionPhase is Field AND Status is Completed 

q322_heatmap = folium.Map([41.90, -87.70], zoom_start = 11)
q322_heatmap.add_child(plugins.HeatMap(location_set322[:40000], radius = 15))

In [24]:
doc323 = {
    'size' : 10000,
    'query': {
        'bool': {
            'must': [{'match': {'DetectionPhase': 'Field'}}, {'match': {'Priority': 'Critical'}}, {'match': {'Status': 'Approved'}}]
        }
    }
}
result323 = es.search(index = 'issues', body = doc323, scroll = '1h')

In [25]:
scroll_size323 = result323['hits']['total']
sid = result323["_scroll_id"]

In [26]:
count = 0
location_set323 = []
while (scroll_size323 > 0):
    print("Scrolling...", count)
    for doc in result323['hits']['hits']:
        location = []
        res = doc['_source']
        count = count + 1
        if 'Latitude' in res.keys():
            print("getting lat")
            if 'Longitude' in res.keys():
                print("getting longi")
                if 'Address' in res.keys():
                    if res['Latitude'] != None and res['Longitude'] != None and res['Address'] != None:
                        print("got one")
                        location.append(float(res['Latitude']))
                        location.append(float(res['Longitude']))
                        location_set323.append(location)
    result323 = es.scroll(scroll_id = sid, scroll = '5m')
    sid = result323['_scroll_id']
    scroll_size323 = len(result323['hits']['hits'])
print("count is: ", count)

Scrolling... 0
getting lat
getting longi
got one
getting lat
getting longi
got one
getting lat
getting longi
count is:  3


In [27]:
location_set323

[[41.917164, -87.686965], [41.891551, -87.607375]]

In [28]:
#Q3.2 3) DetectionPhase is Field AND Priority is Critical AND Status is Approved 

q323_heatmap = folium.Map([41.90, -87.70], zoom_start = 11)
q323_heatmap.add_child(plugins.HeatMap(location_set323[:40000], radius = 15))

In [29]:
doc324 = {
    'size': 10000,
    'query': {
        'bool': {
            'must': [
                     {'match': { 'DetectionPhase': 'Field' }}, 
                     { 'bool' : { 
                         'should': [
                              { 'match': { 'Priority': 'High' }}, { 'match': { 'Priority': 'Critical' }} 
                         ] }
                     }, 
                     { 'bool' : { 
                         'should': [
                              { 'match': { 'Status': 'inProgress' }}, { 'match': { 'Status': 'Approved' }} 
                         ] }
                     }
            ]
        }
    }
}
result324 = es.search(index = 'issues', body = doc324, scroll = '1h')

In [30]:
scroll_size324 = result324['hits']['total']
sid = result324["_scroll_id"]

In [31]:
count = 0
location_set324 = []
while (scroll_size324 > 0):
    print("Scrolling...", count)
    for doc in result324['hits']['hits']:
        location = []
        res = doc['_source']
        count = count + 1
        if 'Latitude' in res.keys():
            print("getting lat")
            if 'Longitude' in res.keys():
                print("getting longi")
                if 'Address' in res.keys():
                    if res['Latitude'] != None and res['Longitude'] != None and res['Address'] != None:
                        print("got one")
                        location.append(float(res['Latitude']))
                        location.append(float(res['Longitude']))
                        location_set324.append(location)
    result324 = es.scroll(scroll_id = sid, scroll = '5m')
    sid = result324['_scroll_id']
    scroll_size324 = len(result324['hits']['hits'])
print("count is: ", count)

Scrolling... 0
getting lat
getting longi
getting lat
getting longi
getting lat
getting longi
getting lat
getting longi
getting lat
getting longi
getting lat
getting longi
getting lat
getting longi
getting lat
getting longi
getting lat
getting longi
getting lat
getting longi
getting lat
getting longi
getting lat
getting longi
getting lat
getting longi
getting lat
getting longi
getting lat
getting longi
getting lat
getting longi
getting lat
getting longi
getting lat
getting longi
getting lat
getting longi
getting lat
getting longi
got one
getting lat
getting longi
got one
getting lat
getting longi
got one
getting lat
getting longi
got one
getting lat
getting longi
count is:  24


In [32]:
location_set324

[[41.853136, -87.63316],
 [41.89323, -87.617419],
 [41.917164, -87.686965],
 [41.891551, -87.607375]]

In [33]:
#Q3.2 4) DetectionPhase is Field AND Priority is Critical or High AND Status is Approved or inProgress 

q324_heatmap = folium.Map([41.90, -87.70], zoom_start = 11)
q324_heatmap.add_child(plugins.HeatMap(location_set324[:40000], radius = 15))

In [34]:
# q3.2 5) 

In [35]:
# Reference: Tutorial on April 13 Video

doc325 = {
    'size':10000,
    'query': {
        "match_all":{}
    },
    "aggs":{
        "selected_dbas":{
            "terms":{
                "field":"Address.keyword",
                "min_doc_count":5,
                "size":10000
                
            },
            "aggs":{
                "top_dba_hits":{
                    "top_hits":{
                        "size":10
                    }
                }
            }
        }
    }
}
result325 = es.search(index = 'issues', body = doc325, scroll = '1h')

In [36]:
scroll_size325 = result325['hits']['total']
sid = result325["_scroll_id"]

In [37]:
result325

{'_scroll_id': 'DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAAFGFklKZEVBUFN6VEplWVVST29xekpZa1EAAAAAAAABRxZJSmRFQVBTelRKZVlVUk9vcXpKWWtRAAAAAAAAAUgWSUpkRUFQU3pUSmVZVVJPb3F6SllrUQAAAAAAAAFJFklKZEVBUFN6VEplWVVST29xekpZa1EAAAAAAAABShZJSmRFQVBTelRKZVlVUk9vcXpKWWtR',
 '_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'aggregations': {'selected_dbas': {'buckets': [{'doc_count': 14,
     'key': '2525 S Martin Luther King Dr',
     'top_dba_hits': {'hits': {'hits': [{'_id': '190',
         '_index': 'issues',
         '_score': 1.0,
         '_source': {'Address': '2525 S Martin Luther King Dr',
          'Address ': None,
          'Author': 'SSP18SCM19P',
          'Category': 'Bug',
          'DetectionPhase': 'Design',
          'Latitude': '41.847095',
          'Latitude ': None,
          'Lattitude': None,
          'Longitude': '-87.616767',
          'Longitude ': None,
          'OriginationPhase': 'Requirements',
          'Priority': 'Critical',
          'State': 'closed',
  

In [38]:
# Reference: Tutorial on April 13 Video

count = 0
location_set325 = []

for doc in result325['aggregations']['selected_dbas']['buckets']:
    location = []
    if 'top_dba_hits' in doc and 'hits' in doc['top_dba_hits'] and 'hits' in doc['top_dba_hits']['hits']:
        
        for hit in doc['top_dba_hits']['hits']['hits']:
            
            if '_source' in hit:
                
                if 'Latitude' in hit['_source'] and 'Longitude' in hit['_source']:
                    if hit['_source']['Latitude'] != None and hit['_source']['Longitude'] != None:
                        location_set325.append([float(hit['_source']['Latitude']), float(hit['_source']['Longitude'])])
                                                

In [39]:
location_set325

[[41.847095, -87.616767],
 [41.847095, -87.616767],
 [41.847095, -87.616767],
 [41.847095, -87.616767],
 [41.847095, -87.616767],
 [41.847095, -87.616767],
 [41.847095, -87.616767],
 [41.847095, -87.616767],
 [41.847095, -87.616767],
 [41.877846, -87.631296],
 [41.877817, -87.631247],
 [41.877817, -87.631247],
 [41.877817, -87.631247],
 [41.877817, -87.631247],
 [41.877817, -87.631247],
 [41.877817, -87.631247],
 [41.877817, -87.631247],
 [41.877817, -87.631247],
 [41.877817, -87.631247],
 [41.883772, -87.625962],
 [41.883772, -87.625962],
 [41.883772, -87.625962],
 [41.883772, -87.625962],
 [41.883772, -87.625962],
 [41.883772, -87.625962],
 [41.883772, -87.625962],
 [41.883772, -87.625962],
 [41.883772, -87.625962],
 [41.883772, -87.625962],
 [41.877846, -87.631296],
 [41.877846, -87.631296],
 [41.877817, -87.631247],
 [41.877817, -87.631247],
 [41.877846, -87.631296],
 [41.877817, -87.631247],
 [41.877846, -87.631296],
 [41.89323, -87.617419],
 [41.89323, -87.617419],
 [41.89323, -8

In [40]:
# q3.2 5) Create Folium HeatMap for all locations that got at least 5 issues create for the same location (latitude, Longitude)

q325_heatmap = folium.Map([41.90, -87.70], zoom_start = 11)
q325_heatmap.add_child(plugins.HeatMap(location_set325[:40000], radius = 15))